In [2]:
import subprocess as sub
import os
import shutil

In [27]:
# User adjustable values

#235U
iso=92235
#energy=[2.53E-8,0.13,0.17,0.3,0.4,0.55,0.58,0.7,0.9,1.0,1.3,1.37,1.7,1.8,2,2.37,4,4.49,4.7,5.5,6,6.3,7.11,\
#8,8.1,9.09,10,11,12.5,13.4,14.0,14.1,14.5,14.8,15.5]
energy=[10]
rundir='/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/E_Dependence/235/FREYA/run/'
outdir='/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/E_Dependence/235/FREYA/data/'

#238U
#iso=92238
#energy=[1.3,1.5,2,2.3,3,3.9,4.49,4.7,4.8,6,6.9,7.1,7.7,8.1,9.1,11.3,13,13.4,14.0,14.1,14.8]
#rundir='/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/E_Dependence/238/FREYA/run/'
#outdir='/home/pyne-user/Dropbox/UCB/Research/ETAs/Scoping_Calcs/FP_Distro/Validation/E_Dependence/238/FREYA/data/'

os.putenv("LD_LIBRARY_PATH","/home/pyne-user/Dropbox/UCB/Computational_Tools/fission_v1.9.2/lib")
os.putenv("FREYADATAPATH","/home/pyne-user/Dropbox/UCB/Computational_Tools/fission_v1.9.2/data_freya")
print len(energy)

1


In [28]:
def freya_input(iso,energy):
    return '#define iterations 1000000\n \
    #define maxA 200\n \
    #define maxN 30       /* The maximum number of ejectiles per fission generated by FREYA */\n \
    \n \
    #include <stdio.h>\n \
    #include <math.h>\n \
    #include "fissionEvent.h"\n \
    \n \
    extern "C" {\n \
       extern int msfreya_setup_();\n \
       extern int msfreya_event_(int,float,float,float*,int*,int*,float*,int*,int*,float*,int*,float*,int*);\n \
       extern int msfreya_getniso_(int *,int *);\n \
       extern int msfreya_getzas_(int *,int *);\n \
       extern float msfreya_sepn_cvt_(int, int, int);\n \
       extern float msfreya_gsmassn_(int, int);\n \
       extern int msfreya_geterrors_(char *, int *);\n \
       extern int msfreya_reseterrorflag_();\n \
       extern bool msfreya_errorflagset_();\n \
    }\n \
    \n \
    void init(void);\n \
    void initFREYA(int& nisosf, int& nisoif, int& niso,\n \
                   int** ZAs, int** fistypes);\n \
    bool FREYA_event(double ePart, int iso, int fissiontype,\n \
                     int*& ZAs, int*& fistypes, int niso,\n \
                     float* tke_dist, int* n_events, int* ff_dist);\n \
    FILE* openfile(char* name);\n \
    void output(float* tke_dist, int* n_events, int* ff_dist);\n \
    \n \
    int main() {\n \
       double energy_MeV = '+str(energy)+';\n \
       int isotope = '+str(iso)+';\n \
       int fissiontype = 1;\n \
    \n \
       int nisosf = 0; // Number of spontaneous fission isotopes\n \
       int nisoif = 0; // Number of induced fission isotopes\n \
       int niso = 0;   // Number of fission isotopes\n \
    \n \
       int** ZAs;      // ZAs of fission isotopes\n \
       int** fistypes; // types of fission [spontaneous (0), induced (1)\n \
       ZAs = new int*;\n \
       fistypes = new int*;\n \
    \n \
       int maxerrorlength=10000;\n \
       char errors[maxerrorlength];\n \
    \n \
       float tke_dist[maxA+1];\n \
       int n_events[maxA+1];\n \
       int ff_dist[maxA+1];\n \
       for (int i=0; i<maxA+1; i++) {\n \
          tke_dist[i] = 0.;\n \
          n_events[i] = 0.;\n \
          ff_dist[i] = 0.;\n \
       }\n \
    \n \
       init();\n\
       initFREYA(nisosf, nisoif, niso, ZAs, fistypes);\n\
       niso=nisosf+nisoif;\n\
       \n\
       for (int i=0; i<iterations; i++) {\n\
          if (!FREYA_event(energy_MeV, isotope, fissiontype, \n\
                           *ZAs, *fistypes, niso,\n\
                           tke_dist, n_events, ff_dist)) {\n\
             int errorlength=maxerrorlength;\n\
             msfreya_geterrors_(&errors[0], &errorlength);\n\
             if (errorlength>1) {\n\
                printf("%s\\n",errors);\n\
                exit(1);\n\
             }\n\
          }\n\
       }\n\
       output(tke_dist, n_events, ff_dist);\n\
    }\n\
    \n\
    FILE* openfile(char* name) {\n\
       FILE* fp = fopen(name, "w");\n\
       if (fp == (FILE *) 0) fprintf(stderr, "Could not open %s for writing\\n", name);\n\
       return fp;\n\
    }\n\
    \n\
    void init(void) {\n\
       unsigned short int s[3] = {1234, 5678, 9012};\n\
       int i;\n\
       seed48(s);\n\
       return;\n\
    }\n\
    \n\
    void initFREYA(int& nisosf, int& nisoif, int& niso,\n\
                   int** ZAs, int** fistypes) {\n\
    \n\
       int maxerrorlength=10000;\n\
       char errors[maxerrorlength];\n\
    \n\
       msfreya_reseterrorflag_();\n\
       msfreya_setup_();\n\
       if (msfreya_errorflagset_()) {\n\
          int errorlength=maxerrorlength;\n\
          msfreya_geterrors_(&errors[0], &errorlength);\n\
          if (errorlength>1) {\n\
             printf("%s\\n",errors);\n\
             exit(1);\n\
          }\n\
       }\n\
       msfreya_getniso_(&nisosf, &nisoif);\n\
       niso=nisosf+nisoif;\n\
    \n\
       // allocate memory to store ZAs for spontaneous and neutron-induced\n\
       // fissions\n\
       *ZAs = new int [niso];\n\
       *fistypes = new int [niso];\n\
    \n\
       // Populate ZAs and fistypes\n\
       msfreya_getzas_(&(*ZAs[0]),&(*fistypes[0]));\n\
    }\n\
    \n\
    bool FREYA_event(double ePart, int iso, int fissiontype,\n\
                     int*& ZAs, int*& fistypes, int niso,\n\
                     float* tke_dist, int* n_events, int* ff_dist\n\
                    ) {\n\
       // if the compound nucleus is ZA, the original nucleus was\n\
       //   ZA for photofission\n\
       //   Z(A-1) for neutron-induced fission\n\
       // treat photofission as if it were neutron-induced fission\n\
       if (fissiontype==2) iso--;\n\
       \n\
       // Find the index of the fission/isotope\n\
       bool foundfission=false;\n\
       int iKm1=0;\n\
       for (iKm1=0; iKm1<niso; iKm1++)\n\
          if (iso == ZAs[iKm1] && ((fissiontype==0) == (fistypes[iKm1]==0))) {\n\
             foundfission=true;\n\
             break;\n\
          }\n\
       if (!foundfission) {\n\
          fprintf(stderr, "ABORT: fission type %d not supported for isotope %d\\n", fissiontype, iso);\n\
          exit(1);\n\
       }\n\
    \n\
       int iK=iKm1+1; // FORTRAN indexing\n\
       int Z=iso/1000;\n\
       int freyaA=iso-1000*Z;\n\
       // watch out! in freya, the A for induced fission is the A of the \n\
       // compound nucleus (for induced fission, add 1 neutron to the nucleus)\n\
       freyaA+=(fissiontype==0)?0:1;\n\
       msfreya_reseterrorflag_();\n\
    \n\
       // Compute nucleus excitation energy for this event\n\
       float eps0;\n\
       float En;\n\
       switch (fissiontype) {\n\
          case 0:\n\
             // spontaneous fission\n\
             eps0 = 0.;\n\
             En=0.;\n\
             break;\n\
          case 1:\n\
             // neutron-induced fission\n\
          case 2:\n\
             // photon-induced fission\n\
             float sepni;\n\
             sepni = msfreya_sepn_cvt_(iK,Z,freyaA);\n\
             if (msfreya_errorflagset_()) return false;\n\
    \n\
             if (fissiontype==1) {\n\
                // neutron-induced fission\n\
                eps0 = sepni+ePart;\n\
                En=ePart;\n\
             } else if (fissiontype==2) {\n\
                // photon-induced fission\n\
                eps0 = ePart;\n\
                En=ePart-sepni;\n\
                if (En<0) En=0.;\n\
             }\n\
             break;\n\
          default:\n\
             fprintf(stderr, "ABORT: fission type %d not supported\\n", fissiontype);\n\
             exit(1);\n\
             break;\n\
       }\n\
    \n\
       // ...generate fission event\n\
       // declare those, msfreya_event needs them\n\
       float V0[3]; // velocity of the initial nucleus\n\
       for (int i=0; i<3; i++) V0[i]=0; // nucleus at rest\n\
    \n\
       float P0[5]; // excited energy, momentum and kinetic energy\n\
                    // of nucleus before interaction\n\
       float P1[5]; // excited energy, momentum and kinetic energy\n\
                    // of fission fragment 1\n\
       float P2[5]; // excited energy, momentum and kinetic energy\n\
                    // of fission fragment 2\n\
       int Z1, A1;  // Charge & mass number of fission fragment 1\n\
       int Z2, A2;  // Charge & mass number of fission fragment 2\n\
    \n\
       float W0=msfreya_gsmassn_(Z, freyaA);  // ground-state mass of nucleus\n\
       if (msfreya_errorflagset_()) return false;\n\
       \n\
       P0[0]=W0+eps0;               // Rest energy of init nucleus\n\
       float g0=1.0;                // gamma0\n\
       P0[4]=g0*P0[0];              // Total energy of init nucleus\n\
       for (int i=0; i<3; i++)\n\
          P0[i]=P0[4]*V0[i];        // Momentum of initial nucleus\n\
    \n\
       int mult;                    // Number of particles emitted\n\
       float particles [4*3*maxN];  // their momentum and kinetic energy\n\
       int ptypes [3*maxN];         // their type: 0(g) & 1(n)\n\
       \n\
       msfreya_event_(iK,En,eps0,&(P0[0]),&Z1,&A1,&(P1[0]),&Z2,&A2,&(P2[0]),&mult,&(particles[0]),&(ptypes[0]));\n\
       if (msfreya_errorflagset_()) return false;\n\
    \n \
       // Get total kinetic energy of fission fragments\n\
       float tot_ke = 0;\n \
       tot_ke += P1[4]-P1[0];\n \
       tot_ke += P2[4]-P2[0];\n \
       \n \
       int AL, AH;      // Mass number of light and heavy fission fragments\n \
       \n \
       AL=A1;\n \
       AH=A2;\n \
       if (AL>AH) {\n \
          AL=A2;\n \
          AH=A1;\n \
       }\n \
       n_events[AH]++;\n \
       tke_dist[AH]+= tot_ke;\n \
       ff_dist[AL]++;\n \
       ff_dist[AH]++;\n \
    \n \
       return true;\n \
    }\n \
    \n \
    void output(float* tke_dist, int* n_events, int* ff_dist) {\n \
       char filename [1024];\n\
    \n \
       sprintf(filename, "tke_versus_AH.res");\n \
       FILE* fp = openfile(filename);\n \
       for (int i=0; i<maxA+1; i++) \n \
          if (n_events[i] != 0)\n \
             fprintf(fp, "%d : %f\\n", i, tke_dist[i]/n_events[i]);\n \
       fclose(fp);\n \
    \n \
       sprintf(filename, "ff_yield.res");\n \
       fp = openfile(filename);\n \
       unsigned int sum=0;\n \
       for (int i=0; i<maxA+1; i++) sum += ff_dist[i];\n \
       for (int i=0; i<maxA+1; i++) fprintf(fp, "%d : %f : %f\\n", i, 1.*ff_dist[i]/sum, 1.*sqrt(ff_dist[i])/sum);\n \
       fclose(fp);\n \
    \n \
       return;\n \
    }'

In [29]:
for e in energy:
    print "Calculating FP distibution for {} w/ {} MeV incident neutrons".format(iso,e)
    # Write FREYA input file
    try: 
        ifile = open(rundir+'src/ff_yield.cpp', 'w') 
        ifile.write("{}\n".format(freya_input(iso,e)))
        # Close the file
        ifile.close()
    except IOError as e:
        print "I/O error({0}): {1}".format(e.errno, e.strerror)

    # Run FREYA
    sub.Popen("rm -rf *",cwd=rundir+"/build",stdout=sub.PIPE,shell=True).communicate()
    sub.Popen("cmake ../src",cwd=rundir+"/build",stdout=sub.PIPE,shell=True).communicate()
    sub.Popen("rm CMakeCache.txt",cwd=rundir+"/build",stdout=sub.PIPE,shell=True).communicate()
    sub.Popen("make",cwd=rundir+"/build",stdout=sub.PIPE,shell=True).communicate()
    print sub.Popen("./ff_yield",cwd=rundir+"/build",stdout=sub.PIPE,shell=True).communicate()

    # Copy output files to data directory
    sub.Popen("rm -rf {}".format(outdir+str(e)),cwd=rundir+"/build",stdout=sub.PIPE,shell=True).communicate()
    os.mkdir(outdir+str(e))
    shutil.copyfile(rundir+'build/ff_yield.res',outdir+str(e)+'/ff_yield.res')
    shutil.copytree(rundir+'plot',outdir+str(e)+'/plot')

Calculating FP distibution for 92235 w/ 10 MeV incident neutrons
('', None)
